In [ ]:
!pip install gradio
!pip install transformers
!pip install gTTS
!pip install pydub
!pip install deep_translator
!pip install pillow
!pip install torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from gtts import gTTS
from pydub import AudioSegment
from deep_translator import GoogleTranslator
from PIL import Image
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/

In [ ]:
# Load the model and transformer
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
def generate_caption(image):
    "Generate image description in English"
    inputs = processor(images=image, return_tensors="pt")
    caption_ids = model.generate(**inputs, max_length=30)
    return processor.decode(caption_ids[0], skip_special_tokens=True)

In [ ]:
def translate_to_arabic(text):
    return GoogleTranslator(source="en", target="ar").translate(text)

In [ ]:
def text_to_speech(text, language):
    file_path = "output.mp3"

    if language == "العربية":
        arabic_intro = "هذا وصف للصورة: "
        arabic_tts = gTTS(text=arabic_intro, lang="ar")
        caption_tts = gTTS(text=text, lang="ar")

        arabic_tts.save("arabic_intro.mp3")
        caption_tts.save("caption.mp3")

        audio1 = AudioSegment.from_mp3("arabic_intro.mp3")
        audio2 = AudioSegment.from_mp3("caption.mp3")
        combined = audio1 + audio2
        combined.export(file_path, format="mp3")
    else:
        tts = gTTS(text=text, lang="en")
        tts.save(file_path)

    return file_path

In [ ]:
def process_image(image_path, language):
    image = Image.open(image_path).convert("RGB")
    caption_en = generate_caption(image)

    if language == "العربية":
        caption_ar = translate_to_arabic(caption_en)
        audio = text_to_speech(caption_ar, language)
        return caption_ar, audio
    else:
        audio = text_to_speech(caption_en, language)
        return caption_en, audio

In [ ]:
import gradio as gr



# Use the background image inside .gradio-container
custom_css = """
.gradio-container {
    background: url('https://tacteec.com/wp-content/uploads/2017/09/2-7.jpg') no-repeat center center fixed;
    background-size: cover;
    color: white;
}

footer, .svelte-1ipelgc {
    background: transparent !important;
}

.block.svelte-1ipelgc {
    background-color: rgba(0, 0, 0, 0.6); /* dark overlay */
    border-radius: 15px;
    padding: 20px;
}
"""

iface = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="filepath", label="Upload an image 📷"),
        gr.Radio(["English", "العربية"], label="Language")
    ],
    outputs=["text", "audio"],
    title="Aisportcaster 🎙",
    description="📷 Upload an image and get a short spoken description in the selected language 🎧",
    css=custom_css
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7eee86dbb4503ebedb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
